# Evaluate KG on the QA task

## Process the groundtruth

In [1]:
import pandas as pd

In [2]:
# Loading the groundtruth
with open(r"C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Evaluation\groundtruth\expert_answers.csv", 'r') as f:
    gtruth = pd.read_csv(f)
gtruth.head()

,QueryID,answer
0,Q1,Membrane Protein (M Protein)
1,Q1,Envelope Protein (E Protein)
2,Q1,Nucleocapsid Protein (N Protein) and Viral RNA
3,Q1,Spike Protein (S Protein)
4,Q2,"Bat,[horseshoe bat]"


In [3]:
# Group the groundtruth by groups
gtruth_group = gtruth.groupby(by = ['QueryID']).agg(lambda x: list(x)).reset_index()
# data3 = data.fillna('').groupby(by = ['QueryID', 'Main_text']).agg(lambda x: list(x)).reset_index()
gtruth_group['length'] = gtruth_group['answer'].apply(lambda x: len(x))
gtruth_group

,QueryID,answer,length
0,Q1,"[Membrane Protein (M Protein), Envelope Protei...",4
1,Q2,"[ Bat,[horseshoe bat], mink, white-tailed de...",5
2,Q3,"[cough, fever, amnesia, headache, myalgia, fat...",23
3,Q4,"[symptomatic treatment, antiviral drug, oxygen...",6
4,Q5,"[tocilizumab, hydroxychloroquine, Acetaminophe...",16


## Process the answer file

In [10]:
class Evaluation:
    
    def __innit__(self, len_ground_truth, retrieved_anws):
        self.len_ground_truth = len_ground_truth
        self.retrieved_anws = retrieved_anws        
        tp = retrieved_anws['label'].value_counts()[1]
        fp = retrieved_anws['label'].value_counts()[0]
        fn = len_ground_truth-retrieved_anws['label'].value_counts()[1]
#         self.precision ()
#         self.recall()
#         self.F1()
    def precision(self, len_ground_truth, retrieved_anws):
        precision = self.tp/(self.tp + self.fp)
        return precision
    def recall (self, len_ground_truth, retrieved_anws):
        recall = self.tp/(self.tp + self.fn)
        return recall
    def F1 (self, len_ground_truth, retrieved_anws):
        f1_score = (2*self.precision*self.recall)/(self.precision+self.recall)
        return f1_score
        
with open(r'C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Results\KG-output-neo4j-2\subset_1\query_1.csv', 'r') as f:
    anws_q1 = pd.read_csv(f)
# anws['label'].value_counts()[1]

len_gold_q1 = gtruth_group[gtruth_group['QueryID']=='Q1']['length'].values[0]
evaluation = Evaluation()
precision_q1 = evaluation.precision(len_gold_q1,anws_q1)
precision_q1

AttributeError: 'Evaluation' object has no attribute 'tp'

In [36]:
# class Evaluation():
#     def __innit__(self, len_ground_truth, retrieved_anws):
#         self.len_ground_truth = len_ground_truth
#         self.retrieved_anws = retrieved_anws
        

def precision(len_ground_truth, retrieved_anws):
    try:
        fp = retrieved_anws['label'].value_counts()[0]
    except:
        fp=0
    tp = retrieved_anws['label'].value_counts()[1]
    fn = len_ground_truth-retrieved_anws['label'].value_counts()[1]
    precision = tp/(tp + fp)
    return precision
def recall (len_ground_truth, retrieved_anws):
    try:
        fp = retrieved_anws['label'].value_counts()[0]
    except:
        fp=0
    tp = retrieved_anws['label'].value_counts()[1]
    fn = len_ground_truth-retrieved_anws['label'].value_counts()[1]
    recall = tp/(tp + fn)
    return recall
def F1 (len_ground_truth, retrieved_anws):
    precision_ = precision(len_ground_truth, retrieved_anws)
    recall_ = precision(len_ground_truth, retrieved_anws)
    f1_score = (2*precision_*recall_)/(precision_+recall_)
    return f1_score


############ Calculate the scores and write them to file #######

n_folders = 10
n_queries = 5
all_scores = []

with open(r'C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Evaluation\result\scores.txt', 'w') as file:
            file.write('folder,query,precision,recall,F1\n')

for i in range (1,n_folders+1):
    query_scores = {}
    for j in range(1,n_queries+1):
        query_scores['query_%s'%j]=[]
        file_p = r"C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Results\KG-output-neo4j-2\subset_{}\query_{}.csv".format(i, j)
        with open(file_p, 'r') as f:
            anws_q = pd.read_csv(f)
        len_gold_q = gtruth_group[gtruth_group['QueryID']=='Q%s'%j]['length'].values[0]
        precision_q = precision(len_gold_q,anws_q)
        recall_q = recall(len_gold_q,anws_q)
        F1_q = F1(len_gold_q,anws_q)
        print('Folder {} -- Query {}: precision: {:.3f}, recall: {:.3f}, F1: {:.3f}\n'.format(i, j, precision_q, recall_q, F1_q))
        query_scores['query_%s'%j].append(precision_q)
        query_scores['query_%s'%j].append(recall_q)
        query_scores['query_%s'%j].append(F1_q)
        with open(r'C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Evaluation\result\scores.txt', 'a') as file:
            file.write('{},{},{:.3f},{:.3f},{:.3f}\n'.format(i, j, precision_q, recall_q, F1_q))

    all_scores.append(folder_scores)


Folder 1 -- Query 1: precision: 0.081, recall: 0.750, F1: 0.081

Folder 1 -- Query 2: precision: 0.455, recall: 1.000, F1: 0.455

Folder 1 -- Query 3: precision: 1.000, recall: 0.043, F1: 1.000

Folder 1 -- Query 4: precision: 0.250, recall: 0.333, F1: 0.250

Folder 1 -- Query 5: precision: 0.250, recall: 0.188, F1: 0.250

Folder 2 -- Query 1: precision: 0.109, recall: 1.750, F1: 0.109

Folder 2 -- Query 2: precision: 0.385, recall: 1.000, F1: 0.385

Folder 2 -- Query 3: precision: 0.250, recall: 0.043, F1: 0.250

Folder 2 -- Query 4: precision: 0.250, recall: 0.667, F1: 0.250

Folder 2 -- Query 5: precision: 0.214, recall: 0.188, F1: 0.214

Folder 3 -- Query 1: precision: 0.087, recall: 1.500, F1: 0.087

Folder 3 -- Query 2: precision: 0.318, recall: 1.400, F1: 0.318

Folder 3 -- Query 3: precision: 0.333, recall: 0.130, F1: 0.333

Folder 3 -- Query 4: precision: 0.300, recall: 1.000, F1: 0.300

Folder 3 -- Query 5: precision: 0.222, recall: 0.250, F1: 0.222

Folder 4 -- Query 1: prec

In [37]:
len(all_scores)

10